## Install Dependencies Required

In [1]:
!pip install numpy
!pip install opencv-python
!pip install matplotlib


In [2]:
import os
import glob as glob
import matplotlib.pyplot as plt
import cv2
import requests
import random
import numpy as np

np.random.seed(42)
TRAIN = True

The dataset is structured in the following manner:

```
├── data.yaml
├── README.dataset.txt
├── README.roboflow.txt
├── test
│   ├── images
│   └── labels
├── train
│   ├── images
│   └── labels
└── valid
    ├── images
    └── labels

```

## Helper function for setting the directory

In [3]:
def set_res_dir():
    # Directory to store results
    res_dir_count = len(glob.glob('runs/train/*'))
    print(f"Current number of result directories: {res_dir_count}")
    if TRAIN:
        RES_DIR = f"results_{res_dir_count+1}"
        print(RES_DIR)
    else:
        RES_DIR = f"results_{res_dir_count}"
    return RES_DIR

## Function for Monitoring Training

In [4]:
def monitor_tensorboard():
    %load_ext tensorboard
    %tensorboard --logdir runs/train --host localhost

## Clone YOLOV5 Repository

In [5]:
if not os.path.exists('yolov5'):
    !git clone https://github.com/ultralytics/yolov5.git

In [6]:
%cd yolov5/
!cd

C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\yolov5
C:\Users\yewyn\Documents\footfall_task\training_data_roboflow\yolov5


In [7]:
!pip install -r requirements.txt

In [8]:
monitor_tensorboard()



Reusing TensorBoard on port 6006 (pid 19080), started 2:12:56 ago. (Use '!kill 19080' to kill it.)

In [ ]:
from yolov5.train import run
RES_DIR = set_res_dir()
run(data='../data.yaml' , weights='yolov5s.pt', img=640, epochs=3, batch_size=16 , name=RES_DIR, optimizer = 'AdamW')
# wandb will ask if you want to visualize your results
# type 3 and enter to get pass it (wandb is deprecated)



wandb: WARNING  wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.


## Helper Functions for inference and extracting frame number

Here we make use of the run function in yolov5's detect.py 

In [ ]:
from detect import run

def inference(RES_DIR, data_path):
    # Directory to store inference results.
    infer_dir_count = len(glob.glob('runs\\detect\\*'))
    INFER_DIR = f"inference_{infer_dir_count+1}"
    print(INFER_DIR)
    # Run inference on images and videos
    run(weights = f"runs\\train\\{RES_DIR}\\weights\\best.pt", source = data_path, name = INFER_DIR, save_txt = True, save_conf = True, conf_thres=0.3)
    return INFER_DIR

In [ ]:
def extract_frame_numbers_and_coordinates(directory):
    frame_coordinates_map = {}
    for file_name in os.listdir(directory):
        if file_name.endswith('.txt'):
            file_path = os.path.join(directory, file_name)
            frame_number = int(file_name.split('_')[1].split('.')[0])
            coordinates = []

            # Read lines from YOLOv5 text file
            with open(file_path, 'r') as file:
                lines = file.readlines()

            # Extract XY coordinates
            for line in lines:
                data = line.strip().split()
                x_center, y_center, width, height = map(float, data[1:5])
                
                # Calculate the coordinates from normalised bounding box to absolute pixel coordinates
                x1 = int((x_center - 0.5 * width) * 960)
                y1 = int((y_center - 0.5 * height) * 720)
                coordinates.append((x1, y1))

            frame_coordinates_map[frame_number] = coordinates

    print(frame_coordinates_map)


## Run Inference on Images collected from the video

In [ ]:
IMAGE_INFER_DIR = inference(RES_DIR, '..\\inferenceImages')

## Run Inference on the initial Video

In [ ]:
VIDEO_INFER_DIR = inference(RES_DIR, '..\\sample.mp4')

## Frame and XY Extraction

Here we use the function we defined earlier to get and store the frame numbers based on the txt files outputed during inference and the xy coordinates into a dictionary form to print out

In [ ]:
# print frames and coordinates where the tag is detected
# the VIDEO_INFER_DIR value represents the path to the latest inference
extract_frame_numbers_and_coordinates(f'.\\runs\\detect\\{VIDEO_INFER_DIR}\\labels')